# Análisis en accidentes de transito

Analizando un DataSet de homicidios en accidentes de transito en Colombia entre los años 2010-2021

In [1]:
#Importamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
#Importamos DataSet
df_accidents = pd.read_csv('DataSet/Homicidios_accidente_de_tr_nsito_Polic_a_Nacional.csv')
df_accidents

FileNotFoundError: [Errno 2] No such file or directory: 'DataSet/Homicidios_accidente_de_tr_nsito_Polic_a_Nacional.csv'

## Preprocesamiento de los datos

In [ ]:
#Renombramos las columnas por comodidad
columns_rename = {column:column.capitalize().replace(' ','_') for column in list(df_accidents)}
df_accidents.rename(columns=columns_rename, inplace=True)

In [ ]:
df_accidents.dtypes

In [ ]:
#Definimos las columnas con valores categorícos
df_accidents[['Departamento', 'Municipio', 
              'Codigo_dane', 'Armas_medios', 
              'Genero', 'Grupo_etarío', 
              'Descripción_conducta']] = df_accidents[['Departamento', 'Municipio', 
                                                       'Codigo_dane', 'Armas_medios', 
                                                       'Genero', 'Grupo_etarío', 
                                                       'Descripción_conducta']].astype('category')

In [ ]:
#Definimos la columna con formato de fecha
df_accidents['Fecha_hecho'] = pd.to_datetime(df_accidents['Fecha_hecho'], format='%d/%m/%Y')

In [ ]:
df_accidents.describe(include='all', datetime_is_numeric=True)

## Procesando Datos

### ¿Cuántos accidentes han ocurrido por año?

In [ ]:
#Copiamos el DataFrame original
df_copy = df_accidents.copy()
df_copy['Total'] = 1

In [ ]:
#Agrupamos por año
fecha_cant_total = df_copy.groupby('Fecha_hecho').sum()
fecha_cant_total_año = fecha_cant_total.resample('Y').sum()
rename_index = {index:index.year for index in fecha_cant_total_año.index}
fecha_cant_total_año = fecha_cant_total_año.rename(index=rename_index)
fecha_cant_total_año

In [ ]:
#Realizamos una representación gráfica de la tabla obtenida anteriormente
año = fecha_cant_total_año.index
cantidad = fecha_cant_total_año['Cantidad'].values

x = np.arange(len(año))

fig, ax = plt.subplots()
ax.grid(linestyle=':')

ax.bar(año, cantidad, width=1, color='salmon', edgecolor="red", linewidth=1)
ax.set_xlabel('Año')
ax.set_ylabel('Accidentes')
ax.set_title('Accidentes entre 2010-2021')
fig.savefig('Imgs/img_accidentes_año.pdf')
fig.savefig('Imgs/img_accidentes_año.jpeg')

plt.show()


Finalemente obtenemos la siguiente tabla:

|Año|Involucrados|Accidentes|
|:--:|:--:|:--:|
|2010|4500|3996|
|2011|4295|3815|
|2012|4549|4076|
|2013|4922|4450|
|2014|4835|4376|
|2015|5344|4888|
|2016|5609|5189|
|2017|5261|4844|
|2018|6017|5532|
|2019|6053|5561|
|2020|5068|4684|
|2021|6914|6323|

- Involucrados: Cantidad de personas en accidentes
- Accidentes: Cantidad de accidentes

### ¿Cuántos accidentes han ocurrido en cada mes?

In [ ]:
#Copiamos el DataFrame original
df_copy_2 = df_accidents.copy()
df_copy_2['Total'] = 1
df_copy_2['Mes'] = df_copy_2['Fecha_hecho'].dt.month.astype('category')
df_copy_2['Año'] = df_copy_2['Fecha_hecho'].dt.year.astype('category')

In [ ]:
df_month_sum = df_copy_2.groupby('Mes').sum()
df_month_sum['Mean'] = df_month_sum['Total'] / 12
df_month_sum

In [ ]:
mes = list(df_month_sum.index)
cantidad_m = df_month_sum['Cantidad'].values

fig, ax = plt.subplots()
ax.grid(linestyle=':')

ax.bar(mes, cantidad_m, width=1, color='salmon', edgecolor="red", linewidth=1)
ax.set_xlabel('Mes')
ax.set_ylabel('Accidentes')
ax.set_title('Accidentes ocurridos por mes entre 2010-2021')
fig.savefig('Imgs/img_accidentes_mes.pdf')
fig.savefig('Imgs/img_accidentes_mes.jpeg')

plt.show()

Obtenemos entonces la siguiente tabla:

|Mes|Involucrados|Accidentes|Promedio|
|:---:|:---:|:---:|:---:|
|Enero|5383|4939|411.58|
|Febrero|4993|4573|381.08|
|Marzo|5306|4793|399.42|
|Abril|4740|4307|358.92|
|Mayo|4996|4536|378.00|
|Junio|5207|4743|395.25|
|Julio|5335|4845|403.75|
|Agosto|5193|4767|397.25|
|Septiembre|5235|4791|399.25|
|Octubre|5582|5121|426.75|
|Noviembre|5318|4819|401.58|
|Diciembre|6079|5500|458.33|

### ¿En qué proporción afecta los accidentes a los grupos etaríos y genero?

In [ ]:
#Copiamos el DataFrame original
df_copy_3 = df_accidents.copy()
pd.get_dummies(df_copy_3['Genero'])
df_copy_3 = pd.concat([df_copy_3, pd.get_dummies(df_copy_3['Genero'])], axis=1)
df_copy_3['Genero'].value_counts()

In [ ]:
df_adultos = df_copy_3.set_index('Grupo_etarío').loc['ADULTOS']
df_adolecentes = df_copy_3.set_index('Grupo_etarío').loc['ADOLESCENTES']
df_menores = df_copy_3.set_index('Grupo_etarío').loc['MENORES']

#### Año

In [ ]:
df_adultos['Total'] = 1
fecha_adultos_año = df_adultos.groupby('Fecha_hecho').sum().resample('Y').sum()
rename_index = {index:index.year for index in fecha_adultos_año.index}
fecha_adultos_año = fecha_adultos_año.rename(index=rename_index)
fecha_adultos_año

In [ ]:
df_adolecentes['Total'] = 1
fecha_adolecentes_año = df_adolecentes.groupby('Fecha_hecho').sum().resample('Y').sum()
rename_index = {index:index.year for index in fecha_adolecentes_año.index}
fecha_adolecentes_año = fecha_adolecentes_año.rename(index=rename_index)
fecha_adolecentes_año

In [ ]:
df_menores['Total'] = 1
fecha_menores_año = df_menores.groupby('Fecha_hecho').sum().resample('Y').sum()
rename_index = {index:index.year for index in fecha_menores_año.index}
fecha_menores_año = fecha_menores_año.rename(index=rename_index)
fecha_menores_año

In [ ]:
año = fecha_adultos_año.index
x = np.arange(len(año))
y_adultos_m = fecha_adultos_año['MASCULINO'].values
y_adultos_f = fecha_adultos_año['FEMENINO'].values

In [ ]:
width = 0.5

#f, (ax1, ax2, x3) = plt.subplots(1, 3, sharey=True)
fig, ax1 = plt.subplots()

bar_am = ax1.bar(año - width/2, y_adultos_m, width=width, label='Masculino')
bar_af = ax1.bar(año + width/2, y_adultos_f, width=width, label='Femenino')
ax1.set(xlim=(2009, 2022))
#ax1.set_xticks(año)
ax1.legend()
ax1.set_xlabel('Año')
ax1.set_ylabel('Accidentes')
ax1.set_title('Accidentes Adultos ocurridos entre 2010-2021')



plt.show()



In [ ]:
año = fecha_adolecentes_año.index
x = np.arange(len(año))
y_adolecentes_m = fecha_adolecentes_año['MASCULINO'].values
y_adolecentes_f = fecha_adolecentes_año['FEMENINO'].values

In [ ]:
fig, ax2 = plt.subplots()

bar_am = ax2.bar(año - width/2, y_adolecentes_m, width=width, label='Masculino')
bar_af = ax2.bar(año + width/2, y_adolecentes_f, width=width, label='Femenino')
ax2.set(xlim=(2009, 2022))
#ax2.set_xticks(año)
ax2.legend()
ax2.set_xlabel('Año')
ax2.set_ylabel('Accidentes')
ax2.set_title('Accidentes adolecentes ocurridos entre 2010-2021')

plt.show()

In [ ]:
año = fecha_menores_año.index
x = np.arange(len(año))
y_menores_m = fecha_menores_año['MASCULINO'].values
y_menores_f = fecha_menores_año['FEMENINO'].values

In [ ]:
fig, ax3 = plt.subplots()

bar_am = ax3.bar(año - width/2, y_menores_m, width=width, label='Masculino')
bar_af = ax3.bar(año + width/2, y_menores_f, width=width, label='Femenino')
ax3.set(xlim=(2009, 2022))
#ax3.set_xticks(año)
ax3.legend()
ax3.set_xlabel('Año')
ax3.set_ylabel('Accidentes')
ax3.set_title('Accidentes menores ocurridos entre 2010-2021')

plt.show()

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3 , sharey=True)

bar_am = ax1.bar(año - width/2, y_adultos_m, width=width, label='Masculino')
bar_af = ax1.bar(año + width/2, y_adultos_f, width=width, label='Femenino')
ax1.set(xlim=(2009, 2022))
##ax1.set_xticks(año)
ax1.legend()
#ax1.set_xlabel('Año')
ax1.set_ylabel('Accidentes')
ax1.set_title('Adultos')

bar_adm = ax2.bar(año - width/2, y_adolecentes_m, width=width, label='Masculino')
bar_adf = ax2.bar(año + width/2, y_adolecentes_f, width=width, label='Femenino')
ax2.set(xlim=(2009, 2022))
##ax2.set_xticks(año)
ax2.legend()
ax2.set_xlabel('Año')
#ax2.set_ylabel('Accidentes')
ax2.set_title('Adolecentes')

bar_mm = ax3.bar(año - width/2, y_menores_m, width=width, label='Masculino')
bar_mf = ax3.bar(año + width/2, y_menores_f, width=width, label='Femenino')
ax3.set(xlim=(2009, 2022))
#0#ax3.set_xticks(año)
ax3.legend()
#ax3.set_xlabel('Año')
#ax3.set_ylabel('Accidentes')
ax3.set_title('Menores')

ax1.grid(linestyle=':')
ax2.grid(linestyle=':')
ax3.grid(linestyle=':')

f.savefig('Imgs/grupos.pdf')

plt.show()